In [1]:
import os


import numpy as np
import pandas as pd
from pandas import DataFrame
import openpyxl
from openpyxl import Workbook
import statsmodels.formula.api as smf

from Constant import Constants as const


In [2]:
# Load data
file_path = os.path.join(const.OUTPUT_PATH, '20241020_cc_reg_data.dta')
data = pd.read_stata(file_path)

# Define dependent variables and control variables
dependent_vars = ['CAPEX_lat_1', 'CAPEX_RDEs_lat_1', 'CAPEX_RDS_lat_1', 'CAPEX_RDI_lat_1']
controls = ['size', 'TobinQ', 'OCF_lat', 'soe', 'lev', 'top1', 'sale_growth']

# Create interaction terms
data['Post2014_has_guarantee'] = data['Post2014'] * data['has_guarantee']
data['Post2017_has_guarantee'] = data['Post2017'] * data['has_guarantee']


C:\Users\wangy\AppData\Local\Temp\ipykernel_13752\971135380.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Post2014_has_guarantee'] = data['Post2014'] * data['has_guarantee']
C:\Users\wangy\AppData\Local\Temp\ipykernel_13752\971135380.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data['Post2017_has_guarantee'] = data['Post2017'] * data['has_guarantee']


In [3]:
# Create an Excel workbook to save the results
output_path = os.path.join(const.OUTPUT_PATH, 'Table3_Baseline_Regression.xlsx')
wb = Workbook()
ws = wb.active
ws.title = "Baseline Regression Results"

# Write headers to the Excel file
headers = [
    "Dependent Variable", "Coefficient", "Std. Error", "t-Statistic", "P-value"
]
ws.append(headers)

# Run regressions and save results to Excel
for dep_var in dependent_vars:
    formula = (
        f"{dep_var} ~ has_guarantee + Post2014 + Post2017 + Post2014_has_guarantee + "
        f"Post2017_has_guarantee + NumGuarantee + TotalLoan + {' + '.join(controls)} + C(tic) + C(year)"
    )
    model = smf.ols(formula=formula, data=data).fit(cov_type='cluster', cov_kwds={'groups': data['tic']})
    print(f"Regression results for {dep_var}:")
    print(model.summary())
    print("\n" + "="*80 + "\n")

    # Write regression results to Excel
    for param in model.params.index:
        ws.append([
            dep_var, param, model.params[param], model.bse[param], model.tvalues[param], model.pvalues[param]
        ])

# Save the Excel workbook
wb.save(output_path)

ValueError: The weights and list don't have the same length.